In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm

import numpy as np

from IPython import embed
from PIL import Image
from skimage import color

import matplotlib.pyplot as plt

import tensorflow as tf

import random

import os

from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from mpl_toolkits.axes_grid1 import ImageGrid

from torchsummary import summary

In [ ]:
dataset_path = "drive/MyDrive/tiny-imagenet-200/"
models_path = 'drive/MyDrive/Università/Magistrale/VCS/models/'
sample_path = "drive/MyDrive/Università/Magistrale/VCS/sample/"

SIZE = 64

In [ ]:
X_test = np.load(dataset_path+'x_test.npy')
Y_test = np.load(dataset_path+'y_test.npy')

In [ ]:
def PSNR(y_true,y_pred):
  return tf.image.psnr(y_true, y_pred, max_val=1.0)

## Utility

In [ ]:
class BaseColor(nn.Module):
    def __init__(self):
        super(BaseColor, self).__init__()

        self.l_cent = 50.
        self.l_norm = 100.
        self.ab_norm = 110.

    def normalize_l(self, in_l):
        return (in_l-self.l_cent)/self.l_norm

    def unnormalize_l(self, in_l):
        return in_l*self.l_norm + self.l_cent

    def normalize_ab(self, in_ab):
        return in_ab/self.ab_norm

    def unnormalize_ab(self, in_ab):
        return in_ab*self.ab_norm

In [ ]:
def load_img(img_path):
    out_np = np.asarray(Image.open(img_path))
    if(out_np.ndim==2):
        out_np = np.tile(out_np[:,:,None],3)
    return out_np

def resize_img(img, HW=(256,256), resample=3):
    return np.asarray(Image.fromarray(img).resize((HW[1],HW[0]), resample=resample))

def preprocess_img(img_rgb_orig, HW=(256,256), resample=3):
    # return original size L and resized L as torch Tensors
    img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)

    img_lab_orig = color.rgb2lab(img_rgb_orig)
    img_lab_rs = color.rgb2lab(img_rgb_rs)

    img_l_orig = img_lab_orig[:,:,0]
    img_l_rs = img_lab_rs[:,:,0]

    tens_orig_l = torch.Tensor(img_l_orig)[None,None,:,:]
    tens_rs_l = torch.Tensor(img_l_rs)[None,None,:,:]

    return (tens_orig_l, tens_rs_l)

def postprocess_tens(tens_orig_l, out_ab, mode='bilinear'):
    # tens_orig_l 1 x 1 x H_orig x W_orig
    # out_ab 1 x 2 x H x W

    HW_orig = tens_orig_l.shape[2:]
    HW = out_ab.shape[2:]

    # call resize function if needed
    if(HW_orig[0]!=HW[0] or HW_orig[1]!=HW[1]):
        out_ab_orig = F.interpolate(out_ab, size=HW_orig, mode='bilinear')
    else:
        out_ab_orig = out_ab

    out_lab_orig = torch.cat((tens_orig_l, out_ab_orig), dim=1)
    return color.lab2rgb(out_lab_orig.data.cpu().numpy()[0,...].transpose((1,2,0)))

## ECCV16

In [ ]:
class ECCVGenerator(BaseColor):
    def __init__(self, norm_layer=nn.BatchNorm2d):
        super(ECCVGenerator, self).__init__()

        model1=[nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[norm_layer(64),]

        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[norm_layer(128),]

        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[norm_layer(256),]

        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[norm_layer(512),]

        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[norm_layer(512),]

        model6=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[norm_layer(512),]

        model7=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[norm_layer(512),]

        model8=[nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]

        model8+=[nn.Conv2d(256, 313, kernel_size=1, stride=1, padding=0, bias=True),]

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model6 = nn.Sequential(*model6)
        self.model7 = nn.Sequential(*model7)
        self.model8 = nn.Sequential(*model8)

        self.softmax = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear')

    def forward(self, input_l):
        conv1_2 = self.model1(self.normalize_l(input_l))
        conv2_2 = self.model2(conv1_2)
        conv3_3 = self.model3(conv2_2)
        conv4_3 = self.model4(conv3_3)
        conv5_3 = self.model5(conv4_3)
        conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv6_3)
        conv8_3 = self.model8(conv7_3)
        out_reg = self.model_out(self.softmax(conv8_3))

        return self.unnormalize_ab(self.upsample4(out_reg))

In [ ]:
def eccv16(pretrained=True):
    model = ECCVGenerator()
    if(pretrained):
        import torch.utils.model_zoo as model_zoo
        model.load_state_dict(
            model_zoo.load_url('https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth',
                               map_location='cpu',
                               check_hash=True)
        )
    return model

In [ ]:
colorizer_eccv16 = eccv16(pretrained=True).eval()

In [ ]:
summary(colorizer_eccv16, (1, 256, 256))

## SIGGRAPH17

In [ ]:
class SIGGRAPHGenerator(BaseColor):
    def __init__(self, norm_layer=nn.BatchNorm2d, classes=529):
        super(SIGGRAPHGenerator, self).__init__()

        # Conv1
        model1=[nn.Conv2d(4, 64, kernel_size=3, stride=1, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[norm_layer(64),]
        # add a subsampling operation

        # Conv2
        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[norm_layer(128),]
        # add a subsampling layer operation

        # Conv3
        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[norm_layer(256),]
        # add a subsampling layer operation

        # Conv4
        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[norm_layer(512),]

        # Conv5
        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[norm_layer(512),]

        # Conv6
        model6=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[norm_layer(512),]

        # Conv7
        model7=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[norm_layer(512),]

        # Conv7
        model8up=[nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=True)]
        model3short8=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]

        model8=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[norm_layer(256),]

        # Conv9
        model9up=[nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=True),]
        model2short9=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        # add the two feature maps above        

        model9=[nn.ReLU(True),]
        model9+=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model9+=[nn.ReLU(True),]
        model9+=[norm_layer(128),]

        # Conv10
        model10up=[nn.ConvTranspose2d(128, 128, kernel_size=4, stride=2, padding=1, bias=True),]
        model1short10=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        # add the two feature maps above

        model10=[nn.ReLU(True),]
        model10+=[nn.Conv2d(128, 128, kernel_size=3, dilation=1, stride=1, padding=1, bias=True),]
        model10+=[nn.LeakyReLU(negative_slope=.2),]

        # classification output
        model_class=[nn.Conv2d(256, classes, kernel_size=1, padding=0, dilation=1, stride=1, bias=True),]

        # regression output
        model_out=[nn.Conv2d(128, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=True),]
        model_out+=[nn.Tanh()]

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model6 = nn.Sequential(*model6)
        self.model7 = nn.Sequential(*model7)
        self.model8up = nn.Sequential(*model8up)
        self.model8 = nn.Sequential(*model8)
        self.model9up = nn.Sequential(*model9up)
        self.model9 = nn.Sequential(*model9)
        self.model10up = nn.Sequential(*model10up)
        self.model10 = nn.Sequential(*model10)
        self.model3short8 = nn.Sequential(*model3short8)
        self.model2short9 = nn.Sequential(*model2short9)
        self.model1short10 = nn.Sequential(*model1short10)

        self.model_class = nn.Sequential(*model_class)
        self.model_out = nn.Sequential(*model_out)

        self.upsample4 = nn.Sequential(*[nn.Upsample(scale_factor=4, mode='bilinear'),])
        self.softmax = nn.Sequential(*[nn.Softmax(dim=1),])

    def forward(self, input_A, input_B=None, mask_B=None):
        if(input_B is None):
            input_B = torch.cat((input_A*0, input_A*0), dim=1)
        if(mask_B is None):
            mask_B = input_A*0

        conv1_2 = self.model1(torch.cat((self.normalize_l(input_A),self.normalize_ab(input_B),mask_B),dim=1))
        conv2_2 = self.model2(conv1_2[:,:,::2,::2])
        conv3_3 = self.model3(conv2_2[:,:,::2,::2])
        conv4_3 = self.model4(conv3_3[:,:,::2,::2])
        conv5_3 = self.model5(conv4_3)
        conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv6_3)

        conv8_up = self.model8up(conv7_3) + self.model3short8(conv3_3)
        conv8_3 = self.model8(conv8_up)
        conv9_up = self.model9up(conv8_3) + self.model2short9(conv2_2)
        conv9_3 = self.model9(conv9_up)
        conv10_up = self.model10up(conv9_3) + self.model1short10(conv1_2)
        conv10_2 = self.model10(conv10_up)
        out_reg = self.model_out(conv10_2)

        conv9_up = self.model9up(conv8_3) + self.model2short9(conv2_2)
        conv9_3 = self.model9(conv9_up)
        conv10_up = self.model10up(conv9_3) + self.model1short10(conv1_2)
        conv10_2 = self.model10(conv10_up)
        out_reg = self.model_out(conv10_2)

        return self.unnormalize_ab(out_reg)

In [ ]:
def siggraph17(pretrained=True):
    model = SIGGRAPHGenerator()
    if(pretrained):
        import torch.utils.model_zoo as model_zoo
        model.load_state_dict(model_zoo.load_url('https://colorizers.s3.us-east-2.amazonaws.com/siggraph17-df00044c.pth',map_location='cpu',check_hash=True))
    return model

In [ ]:
colorizer_siggraph17 = siggraph17(pretrained=True).eval()

In [ ]:
summary(colorizer_siggraph17, (1, 256, 256))

## Test metrics

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ECCV16_SSIM = 0
ECCV16_PSNR = 0
SIGGRAPH17_SSIM = 0
SIGGRAPH17_PSNR = 0

size = len(X_test)
for i in tqdm(range(len(X_test[:size]))):   

    # image reconstruction
    L = X_test[i]
    AB = Y_test[i]
    
    img = np.zeros((SIZE, SIZE, 3))
    img[:,:,0] = L[:,:,0]
    img[:,:,1:] = AB*128
    img = lab2rgb(img)
    img = (img * 255).astype(np.uint8)

    # preprocessing
    (tens_l_orig, tens_l_rs) = preprocess_img(img, HW=(64,64))

    img_bw = postprocess_tens(tens_l_orig, torch.cat((0*tens_l_orig,0*tens_l_orig),dim=1))
    
    # colorization
    eccv16_colorized = colorizer_eccv16(tens_l_rs).cpu()
    siggraph17_colorized = colorizer_siggraph17(tens_l_rs).cpu()

    # postprocessing
    out_img_eccv16 = postprocess_tens(tens_l_orig, eccv16_colorized)
    out_img_siggraph17 = postprocess_tens(tens_l_orig, siggraph17_colorized)

    # metrics computation
    
    img_or = tf.image.convert_image_dtype(img, tf.float32)
    im1 = tf.image.convert_image_dtype(out_img_eccv16, tf.float32)
    im2 = tf.image.convert_image_dtype(out_img_siggraph17, tf.float32)

    ssim1 = tf.image.ssim(img_or, im1, max_val=1.0, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    ssim2 = tf.image.ssim(img_or, im2, max_val=1.0, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    
    psnr1 = PSNR(img_or,im1)
    psnr2 = PSNR(img_or,im2)
    
    ECCV16_SSIM += ssim1
    SIGGRAPH17_SSIM += ssim2

    ECCV16_PSNR += psnr1
    SIGGRAPH17_PSNR += psnr2

print(f"ECCV16 --> SSIM {ECCV16_SSIM/size} - PSNR {ECCV16_PSNR/size}")
print(f"SIGGRAPH17 --> SSIM {SIGGRAPH17_SSIM/size} - PSNR {SIGGRAPH17_PSNR/size}")
# ECCV16 --> SSIM 0.9230453372001648 - PSNR 22.14032745361328
# SIGGRAPH17 --> SSIM 0.9334288239479065 - PSNR 23.900009155273438

## Colorization comparison

In [ ]:
polychromify = tf.keras.models.load_model(models_path+"Polychromify", custom_objects={'PSNR':PSNR})
polychromifyA = tf.keras.models.load_model(models_path+"Polychromify_A", custom_objects={'PSNR':PSNR})
polychromifyB = tf.keras.models.load_model(models_path+"Polychromify_B", custom_objects={'PSNR':PSNR})

In [ ]:
outputs = []
indexes = [3,4,18,8167,691,6956]
left = 6-len(indexes)
for i in range(left):
  indexes.append(round(random.random()*len(X_test)))

size = len(indexes)
print(indexes)

for i in indexes:       

    # image merging
    L = X_test[i]
    AB = Y_test[i]
    x = X_test[i]
    
    img = np.zeros((SIZE, SIZE, 3))
    img[:,:,0] = L[:,:,0]
    img[:,:,1:] = AB*128
    img = lab2rgb(img)
    img = (img * 255).astype(np.uint8)

    # preprocessing
    (tens_l_orig, tens_l_rs) = preprocess_img(img, HW=(64,64))
    img_bw = postprocess_tens(tens_l_orig, torch.cat((0*tens_l_orig,0*tens_l_orig),dim=1))
    
    # colorization
    eccv16_colorized = colorizer_eccv16(tens_l_rs).cpu()
    siggraph17_colorized = colorizer_siggraph17(tens_l_rs).cpu()    

    img_color = np.array([x], dtype=float)

    # polychromify    
    output = polychromify.predict(img_color)
    result = np.zeros((SIZE, SIZE, 3))
    result[:,:,0] = x[:,:,0]
    result[:,:,1:] = output[0]*128
    out_img_polychromify = lab2rgb(result)

    # polychromifyAB    
    outputA = polychromifyA.predict(img_color)
    outputB = polychromifyB.predict(img_color)
    result = np.zeros((SIZE, SIZE, 3))
    result[:,:,0] = x[:,:,0]
    result[:,:,1] = outputA[0][:,:,0]*128
    result[:,:,2] = outputB[0][:,:,0]*128
    out_img_polychromifyAB = lab2rgb(result)

    ##### ONLY FOR ECCV16 & SIGGRAPH17 #####

    # postprocessing
    out_img_eccv16 = postprocess_tens(tens_l_orig, eccv16_colorized)
    out_img_siggraph17 = postprocess_tens(tens_l_orig, siggraph17_colorized)
    # resize
    out_img_eccv16 =  resize(out_img_eccv16, (SIZE,SIZE))
    out_img_siggraph17 = resize(out_img_siggraph17, (SIZE,SIZE))

    ### metrics computation ###

    img = tf.image.convert_image_dtype(img, tf.float32)
    im_ECCV16 = tf.image.convert_image_dtype(out_img_eccv16, tf.float32)
    im_SIGGRAPH17 = tf.image.convert_image_dtype(out_img_siggraph17, tf.float32)
    im_POLY = tf.image.convert_image_dtype(out_img_polychromify, tf.float32)
    im_POLYAB = tf.image.convert_image_dtype(out_img_polychromifyAB, tf.float32)

    ssim_ECCV16 = tf.image.ssim(img, im_ECCV16, max_val=1.0, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    ssim_SIGGRAPH17 = tf.image.ssim(img, im_SIGGRAPH17, max_val=1.0, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    ssim_POLY = tf.image.ssim(img, im_POLY, max_val=1.0, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    ssim_POLYAB = tf.image.ssim(img, im_POLYAB, max_val=1.0, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    
    psnr_ECCV16 = PSNR(img,im_ECCV16)
    psnr_SIGGRAPH17 = PSNR(img,im_SIGGRAPH17)
    psnr_POLY = PSNR(img,im_POLY)
    psnr_POLYAB = PSNR(img,im_POLYAB)

    ######################################################################## todo: save these metrics
    # print(f"ECCV16 --> SSIM {ssim_ECCV16} - PSNR {psnr_ECCV16}")
    # print(f"SIGGRAPH17 --> SSIM {ssim_SIGGRAPH17} - PSNR {psnr_SIGGRAPH17}")
    # print(f"POLY --> SSIM {ssim_POLY} - PSNR {psnr_POLY}")
    # print(f"POLYAB --> SSIM {ssim_POLYAB} - PSNR {psnr_POLYAB}")

    outputs.append(img_bw)
    outputs.append(out_img_eccv16)
    outputs.append(out_img_siggraph17)
    outputs.append(out_img_polychromify)
    outputs.append(out_img_polychromifyAB)    
    outputs.append(img)    

In [ ]:
titles = ["Input","ECCV16","SIGGRAPH17","Polychromify","PolychromifyAB","Original"]

fig = plt.figure(figsize=(15, 15))
grid = ImageGrid(fig, 111, nrows_ncols=(size,len(titles)), axes_pad=0)

counter = 0
for ax, im in zip(grid, outputs):    
    ax.imshow(im)
    ax.set_yticklabels([])
    ax.set_xticklabels([])        
    if counter < len(titles):
      ax.set_title(titles[counter])    
    counter+=1
plt.show()